In [7]:
from langchain_core.prompts import ChatPromptTemplate

classify_request_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객의 문의에 대응되는 요청을 판단하는 로봇이야.
            사용자가 입력한 메시지를 보고 아래 요청 유형 중 하나로 분류해야 해.
            -상품 주문에 관한 내용: '주문 완료'
            -주문 변경에 관한 문의: '주문 변경'
            -주문 취소에 관한 문의: '주문 취소'
            """
        ),
        ("human", "{input}"),

    ]
)

In [27]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [28]:
from langchain_core.runnables import RunnablePassthrough

classify_request_chain = RunnablePassthrough.assign(inquiry_type=classify_request_prompt | model)

In [29]:
from pydantic import BaseModel, Field, conint, condecimal
from typing import List

class OrderItemData(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    quantity: conint(gt=0) = Field(..., description="Quantity of the product")
    price: condecimal(max_digits=10, decimal_places=2) = Field(..., description="Price of the product")

class CreateOrderData(BaseModel):
    user_id: int = Field(..., description="ID of the user creating the order")
    items: List[OrderItemData] = Field(..., description="List of items to order")

class UpdateOrderStatusData(BaseModel):
    order_id: int = Field(..., description="ID of the order to update")
    new_status: str = Field(..., description="New status of the order")

In [30]:
parsers = [OrderItemData, CreateOrderData, UpdateOrderStatusData]

In [31]:

model_with_parsers = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools(parsers)

In [32]:
model_with_parsers.kwargs

{'tools': [{'type': 'function',
   'function': {'name': 'OrderItemData',
    'description': "Usage docs: https://docs.pydantic.dev/2.7/concepts/models/ A base class for creating Pydantic models. Attributes:\n    __class_vars__: The names of classvars defined on the model.\n    __private_attributes__: Metadata about the private attributes of the model.\n    __signature__: The signature for instantiating the model.     __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.\n    __pydantic_core_schema__: The pydantic-core schema used to build the SchemaValidator and SchemaSerializer.\n    __pydantic_custom_init__: Whether the model has a custom `__init__` function.\n    __pydantic_decorators__: Metadata containing the decorators defined on the model.\n        This replaces `Model.__validators__` and `Model.__root_validators__` from Pydantic V1.\n    __pydantic_generic_metadata__: Metadata for generic models; contains data used for a similar pur

In [33]:
model_with_parsers.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'OrderItemData',
   'description': "Usage docs: https://docs.pydantic.dev/2.7/concepts/models/ A base class for creating Pydantic models. Attributes:\n    __class_vars__: The names of classvars defined on the model.\n    __private_attributes__: Metadata about the private attributes of the model.\n    __signature__: The signature for instantiating the model.     __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.\n    __pydantic_core_schema__: The pydantic-core schema used to build the SchemaValidator and SchemaSerializer.\n    __pydantic_custom_init__: Whether the model has a custom `__init__` function.\n    __pydantic_decorators__: Metadata containing the decorators defined on the model.\n        This replaces `Model.__validators__` and `Model.__root_validators__` from Pydantic V1.\n    __pydantic_generic_metadata__: Metadata for generic models; contains data used for a similar purpose to\n   

In [34]:
import json
import os

current_dir = os.getcwd()

# JSON 파일 경로
json_file_path = os.path.join(current_dir, 'products.json')

# JSON 파일을 읽어와서 파싱
with open(json_file_path, 'r', encoding='utf-8') as file:
    products = json.load(file)

# 상품 정보 출력
for product in products:
    print(f"상품: {product['product_name']}, 기본 판매 수량: {product['quantity']}개, 기본 판매 수량의 가격: {product['price']}원")


상품: 떡케익5호, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 54000원
상품: 무지개 백설기 케익, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 51500원
상품: 미니 백설기, 기본 판매 수량: 35개, 기본 판매 수량의 가격: 31500원
상품: 개별 모듬팩, 기본 판매 수량: 1개, 기본 판매 수량의 가격: 13500원


In [35]:
products

[{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000},
 {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500},
 {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500},
 {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}]

In [36]:
from langchain_core.prompts import ChatPromptTemplate

classify_request_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문 관련 요청을 아래 분류하고, 각 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.
            주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'
            
            상품 정보
            {products}
            

            user_id: {user_id}
            """
        ),
        ("human", "{input}"),

    ]
)

In [37]:
from langchain_core.runnables import RunnablePassthrough

classify_request_chain = RunnablePassthrough.assign(inquiry_type=classify_request_prompt | model)

In [38]:
classify_request_chain = classify_request_prompt | model_with_parsers 
classify_request_chain

ChatPromptTemplate(input_variables=['input', 'products', 'user_id'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['products', 'user_id'], template="\n            너는 주문 관련 요청을 아래 분류하고, 각 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.\n            주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'\n            \n            상품 정보\n            {products}\n            \n\n            user_id: {user_id}\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002D1F4F56440>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002D1F4F57370>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'OrderItemData', 'description': "Usage docs: https://docs.pydantic.dev/2.7/concepts/models/ A base class for

In [39]:
classify_request_chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[0].function.description': string too long. Expected a string with maximum length 1024, but got a string with length 1824 instead.", 'type': 'invalid_request_error', 'param': 'tools[0].function.description', 'code': 'string_above_max_length'}}

In [41]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([CreateOrderData])
chain = classify_request_prompt | model 
chain

ChatPromptTemplate(input_variables=['input', 'products', 'user_id'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['products', 'user_id'], template="\n            너는 주문 관련 요청을 아래 분류하고, 각 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.\n            주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'\n            \n            상품 정보\n            {products}\n            \n\n            user_id: {user_id}\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002D1F51C0F40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002D1F51C09A0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'CreateOrderData', 'description': "Usage docs: https://docs.pydantic.dev/2.7/concepts/models/ A base class f

In [42]:
chain.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools[0].function.description': string too long. Expected a string with maximum length 1024, but got a string with length 1824 instead.", 'type': 'invalid_request_error', 'param': 'tools[0].function.description', 'code': 'string_above_max_length'}}

---

In [58]:
template = """
너는 주문 관련 요청을 아래 분류하고, 각 요청 처리 함수에 필요한 인자를 추출하는 로봇이야.
주문 관련 요청은 다음 중 하나로 분류해야 해: '주문 요청', '주문 변경 요청', '주문 취소 요청'

상품 정보
{products}

user_id: {user_id}


user input: {input}
"""

In [59]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=CreateOrderData)

In [60]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["uesr_id", "input", "products"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [61]:
model = ChatOpenAI()

In [62]:
prompt_and_model = prompt | model
prompt_and_model

PromptTemplate(input_variables=['input', 'products', 'user_id'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"OrderItemData": {"properties": {"product_name": {"description": "Name of the product", "title": "Product Name", "type": "string"}, "quantity": {"description": "Quantity of the product", "exclusiveMinimum": 0, "title": "Quantity", "type": "integer"}, "price": {"anyOf": [{"type": "number"}, {"type": "string"}], "description": "Price of the product", "title": "Price"}}, "required": ["product_name", "quantity", "price"], "

In [64]:
response = prompt_and_model.invoke(
    {"user_id": 1,
     "input": "개별 모듬팩 2개 주문할게요",
     "products": products}
)
response

AIMessage(content="주문 요청\n인자: user_id=1, product_name='개별 모듬팩', quantity=2", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 248, 'total_tokens': 277}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b0ad5b70-402c-4a0e-86e7-f7d25d2d1b44-0')

In [66]:
print(response.content)

주문 요청
인자: user_id=1, product_name='개별 모듬팩', quantity=2
